In [1]:
import pandas as pd
import numpy as np
import ast

In [17]:
dset= pd.read_csv('dhaka_div_updated.csv')

In [4]:
dset.columns

Index(['osm_type', 'name', 'category', 'type', 'lat', 'lon', 'tags'], dtype='object')

In [12]:
dset.drop('id',axis = 1, inplace=True)
dset.head()

,osm_type,name,category,type,lat,lon,tags
0,node,German Club - Rest House Society,amenity,pub,23.793577,90.419754,"{'addr:city': 'Dhaka', 'addr:country': 'BD', '..."
1,node,American Club - AEEA,amenity,pub,23.802524,90.410248,"{'amenity': 'pub', 'name': 'American Club - AE..."
2,node,Netherland Recreation Center (Dutch Club),amenity,pub,23.803080,90.413906,"{'amenity': 'pub', 'name': 'Netherland Recreat..."
3,node,NaN,amenity,ferry_terminal,23.894661,90.571004,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ..."
4,node,NaN,amenity,ferry_terminal,23.621013,90.452398,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ..."


In [22]:
# print(type(dset['tags'][12]))
# x = ast.literal_eval(dset['tags'][12])
# dset['tags'][12]=x
# print(type(dset['tags'][12]))
# currently all data is in str form so it needs to convern into dict
# dset['tags'] = dset['tags'].apply(ast.literal_eval)
def safe_eval(v):
    if isinstance(v, str):
        try:
            return ast.literal_eval(v)
        except:
            return v   
    return v  
dset['tags'] = dset['tags'].apply(safe_eval)

In [55]:
# dset['type'].isna()[8] #np.True_
# pd.isna(dset.loc[8, 'type']) #True
dset['type'].isna().sum()

np.int64(5312)

# Data AUG

In [75]:
for i in range(len(dset)):
    if pd.isna(dset.loc[i, 'type']):
        x = dset.loc[i, 'tags']  
        if 'railway' in x:
            dset.loc[i, 'type'] = 'railway'
            if pd.isna(dset.loc[i, 'name']):
                for key in ['name', 'name:en', 'name:bn']:
                    if key in x:
                        dset.loc[i, 'name'] = x[key]
                        break  

In [9]:
for i in range(len(dset)):
    if pd.isna(dset.loc[i, 'name']):
        x = dset.loc[i, 'tags']
        for key in ['name', 'name:en', 'name:bn']:
            if key in x:
                dset.loc[i, 'name'] = x[key]
                break

In [26]:
dset.to_csv('dhaka_div_updated.csv', index=False)

In [30]:
#  to drop specefic row based on tags
# rows_to_drop = [
#     i for i, row in dset.iterrows()
#     if pd.isna(row['name']) and pd.isna(row['type']) and 'man_made' in row['tags']
# ]
# dset.drop(rows_to_drop, inplace=True)
# dset.reset_index(drop=True, inplace=True)
rows_to_drop = [
    i for i, row in dset.iterrows()
    if pd.isna(row['name']) and pd.isna(row['type']) and 'aeroway' in row['tags']
]
dset.drop(rows_to_drop, inplace=True)
dset.reset_index(drop=True, inplace=True)

In [20]:
duplicates = dset[dset.duplicated(subset=['lat','lon','type','name'], keep=False)]
duplicate_indexes = duplicates.index.tolist()
print("Number of duplicate rows:", len(duplicate_indexes))
print("Indexes of duplicate rows:", duplicate_indexes)

Number of duplicate rows: 10
Indexes of duplicate rows: [4626, 6584, 12493, 12737, 15282, 15308, 26448, 26454, 30754, 31158]


In [24]:
dset.drop(duplicate_indexes, inplace=True)
dset.reset_index(drop=True, inplace=True)

In [3]:
len(dset)

40373

In [40]:
list_of_both_miss = []

for i, row in dset.iterrows():
    if pd.isna(row['name']) and pd.isna(row['type']):
        list_of_both_miss.append(i)
        
dd=dset.loc[list_of_both_miss]
# after all of preproseeing there only 165 rows , so  those are erased
# dset.drop(list_of_both_miss, inplace=True)
# dset.reset_index(drop=True, inplace=True)

,osm_type,name,category,type,lat,lon,tags


In [4]:
dset['name'].isna().sum()

np.int64(8883)

In [25]:
dset['type'].isna().sum()

np.int64(119)

In [28]:
# for index, row in dset.iterrows():
#     if pd.isna(row['type']):
#         print(f"Name     : {row['name']}")
#         print(f"Category : {row['category']}")
#         print(f"Tags     : {row['tags']}")

# priority_keys = ["historic", "man_made", "aeroway"]

# for index, row in dset.iterrows():
#     if pd.isna(row["type"]):
#         tags = row["tags"]
#         for key in priority_keys:
#             if key in tags:
#                 dset.at[index, "type"] = tags[key]
#                 break  

# for index, row in dset.iterrows():
#     if pd.isna(row["type"]):
#         tags = row["tags"]

#         if "craft" in tags:
#             dset.at[index, "type"] = tags["craft"]

In [29]:
dset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40373 entries, 0 to 40372
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   osm_type  40373 non-null  object 
 1   name      31490 non-null  object 
 2   category  40373 non-null  object 
 3   type      40254 non-null  object 
 4   lat       40373 non-null  float64
 5   lon       40373 non-null  float64
 6   tags      40373 non-null  object 
dtypes: float64(2), object(5)
memory usage: 2.2+ MB
